In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np
from matplotlib.pyplot import plot, savefig
import warnings
warnings.filterwarnings('ignore')


---

## Funkcje 

In [2]:
def stance_analysis(file:str,
                    name_save:str='figure', name_author:str='polityk', 
                    if_sejm:bool=False
                    ):
        
    data = pd.read_pickle(f'{file}')
    data['stance'] = data['stance'].replace({'NEITHER': 0,
                                    'FAVOR': 1,
                                    'AGAINST': -1,
                                    })
    
    if 'author' not in data.columns:
        data.rename(columns={'username': 'author'}, inplace=True)

    if if_sejm:
        data['author'] = data['author'].apply(lambda x: x.title())

    # średnia z dnia dla stance 
    data['date'] = data['date'].apply(pd.to_datetime).dt.date
    df_mean_stance = data.groupby(['date', 'topic_name', 'author'])['stance'].mean().reset_index(name='mean_stance') # jeżeli w dniu więcej jak jedna zmiana to wtedy ma wpływ 
    df_mean_stance.sort_values(['author', 'topic_name', 'date'], ascending=True, inplace=True) 
    df_mean_stance.reset_index(drop=True, inplace=True)

    # czy w danym temacie u danego autora nastąpiłą zmiana stance z -1 na 1 lub na odwrót?
    df_mean_stance['stance_changed_author_topic'] = df_mean_stance.groupby(by=['author', 'topic_name'])['mean_stance'].transform(lambda x: x.max() > 0 and x.min() < 0)


    # --------------------------------------------------------------------------------------------------------
    # obliczenie dni pomiędzy zminaą mean_stance z -1 na 1 lub na odwród dla danego autora w danym temacie + dodanie kolumny stance_change, wpisujemy True jeżeli taka zmiana nastąpiła 
    
    df_stance_mean_changed = df_mean_stance[df_mean_stance['stance_changed_author_topic'] == True] #wsystkie tematy z autorami gdzie autor zmienił stance (zmiana stance_mean)
    df_stance_mean_changed.reset_index(drop=True, inplace=True)
    df_stance_mean_changed['count_days'] = pd.NA
    authors = df_stance_mean_changed['author'].unique()

    for author in authors:
        counter = 0
        for i in df_stance_mean_changed[df_stance_mean_changed['author'] == author].index:
            if counter == 0:
                counter += 1
                n = i 
            else:
                if df_stance_mean_changed.iloc[i]['topic_name'] == df_stance_mean_changed.iloc[i-1]['topic_name']:
                    if df_stance_mean_changed.iloc[i]['mean_stance'] * df_stance_mean_changed.iloc[n]['mean_stance'] < 0: 
                        df_stance_mean_changed.at[i, 'count_days'] = (df_stance_mean_changed.iloc[i]['date'] - df_stance_mean_changed.iloc[n]['date']).days
                        df_stance_mean_changed.at[i, 'stance_changed'] = True
                        n = i
                    else:
                        if df_stance_mean_changed.iloc[i]['mean_stance'] != 0 and df_stance_mean_changed.iloc[n]['mean_stance']==0:
                            n=i
                        df_stance_mean_changed.at[i, 'stance_changed'] = False
                else:
                    n = i
    
    df_mean_stance = pd.merge(df_mean_stance, df_stance_mean_changed[["date", "topic_name", "author", "stance_changed", "count_days"]], on=["date", "topic_name", "author"], how='left')
    df_mean_stance['stance_changed'] = df_mean_stance['stance_changed'].fillna(False)


    # --------------------------------------------------------------------------------------------------------
    # wyliczenie procenta wypowiedzi według tematów, gdzie nastąpiła zmiana nastawienia 

    print('\n\nW których tematach najczęściej zmieniano zdanie?\n')
    df_mean_stance['count_change'] = df_mean_stance[df_mean_stance['stance_changed'] == True].groupby(by=['topic_name'])['topic_name'].transform('count') # zlicza liczbę tweetów w danym temacie gdzie wystąpiła zmiana zdania 
    df_mean_stance['count_topic'] = df_mean_stance.groupby(by=['topic_name'])['stance_changed'].transform('count') # zlicza liczbę wszytkich tweetów w danym temacie

    
    df_mean_stance['percent_stance_change'] = df_mean_stance['count_change'] / df_mean_stance['count_topic']
    df_mean_stance['percent_stance_change'] *= 100
    

    df_final = df_mean_stance[df_mean_stance['stance_changed']==True].drop_duplicates(subset=['topic_name']).sort_values('percent_stance_change', ascending=False).reset_index(drop=True)

    df_final_10 = df_final.sort_values('percent_stance_change', ascending=False).head(10)
    color_discrete_sequence = ['#414fa0']*len(df_final)
    
    fig = px.bar(df_final_10, x='percent_stance_change', y='topic_name', 
                 labels={'percent_stance_change': 'Procent wypowiedzi, gdzie nastąpiła zmiana nastawienia [%]', 'topic_name': 'Temat'}, color_discrete_sequence=color_discrete_sequence,
                 width=850, height=350)
    
    fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)'
    })

    # zapis
    pio.write_image(fig, f"{name_save}_topic_stance_percent.svg")
    fig.show()

    # ------------------------------------------------------------------------------------------------------------
    # zliczenie dla każdego autora w ilu wypowiedziach zmienił nastawienie 
    
    print('\n\nKto najczęściej zmienia swoje poglądy?\n')
    df_mean_stance['count_change_author'] = df_mean_stance[df_mean_stance['stance_changed'] == True].groupby(by=['author'])['topic_name'].transform('count') # zlicza liczbę tweetów w danym temacie gdzie wystąpiła zmiana zdania 
    df_final_author = df_mean_stance[df_mean_stance['stance_changed']==True].drop_duplicates(subset=['author']).sort_values('count_change_author', ascending=False).reset_index(drop=True)

    display(df_final_author[['author', 'count_change_author']])

    df_final_10_author = df_final_author.head(10)
    color_discrete_sequence = ['#414fa0']*len(df_final_10_author)
    
    fig = px.bar(df_final_10_author, x='count_change_author', y='author', 
                 labels={'count_change_author': 'Liczba wypowiedzi, gdzie nastąpiła zmiana zdania', 'author': name_author}, color_discrete_sequence=color_discrete_sequence,
                 width=750, height=350)
    fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)'
    })

    # zapis
    pio.write_image(fig, f"{name_save}_author_change_stance_count.svg")
    fig.show()

    # -----------------------------------------------------------------------------------
    # obliczanie średniej liczby dni dla danego autora w danym temacie, następnie obliczenie średnich dla danego tematu po średnich autorów
    print('\n\nIle czasu średnio potrzeba na zmianę poglądu?\n')
    
    days_per_author_topic = df_mean_stance.dropna(subset=['count_days'])
    days_per_author_topic = days_per_author_topic.groupby(['author', 'topic_name'])['count_days'].mean().reset_index()
    days_per_author_topic = days_per_author_topic.rename(columns={'count_days': 'mean_days_author_topic'})

    days_per_topic = days_per_author_topic.groupby(['topic_name'])['mean_days_author_topic'].mean().reset_index()
    days_per_topic = days_per_topic.rename(columns={'mean_days_author_topic': 'mean_days_topic'})

    days_per_topic = days_per_topic.sort_values('mean_days_topic', ascending=False)
    days_per_topic.dropna(subset=['mean_days_topic'], inplace=True)
    days_per_topic_10 = days_per_topic.head(10)
    days_per_topic_10 = days_per_topic_10.sort_values('mean_days_topic', ascending=True)
    display(days_per_topic_10)
    fig = px.bar(days_per_topic_10, y='topic_name', x='mean_days_topic', orientation='h',
                color_discrete_sequence=color_discrete_sequence,
                labels={'topic_name': 'Temat', 'mean_days_topic': 'Średnia liczba dni'}, width=750, height=350)
    fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)'
    })
    pio.write_image(fig, f"{name_save}_mean_days_topic.svg")
    fig.show()

    # print(f"Potrzeba średnio {np.mean(df_mean_stance['count_days'].dropna().to_list())} dni na zmianę poglądów - bez podziałów")
    # print(f"Potrzeba średnio {np.mean(days_per_author_topic['mean_days_author_topic'].to_list())} dni na zmianę poglądów - podział na tematy")
    print(f"Potrzeba średnio {np.mean(days_per_topic['mean_days_topic'].dropna().to_list())} dni na zmianę poglądu (średnia  z autorami + temat, a następnie średnia po temacie")

    return df_mean_stance
    


In [3]:
def topic_analysis(data:pd.DataFrame,
                   name_save:str = 'figure', author:str='Mówca'):
    
    if 'author' not in data.columns:
        data.rename(columns={'username': 'author'}, inplace=True)

    data_topic_grouped = data.groupby(['author', 'topic_name'])['date'].count().reset_index(name='count').sort_values('count', ascending=False) 
    
    data_topics_per_author = data.drop_duplicates(subset=['author', 'topic_name'])
    data_topics_per_author = data_topics_per_author.groupby(['author'])['topic_name'].count().reset_index(name='count_topics').sort_values('count_topics', ascending=False) 
    top_active_authors = data_topics_per_author.head(10)
    
    print('Kto porusza najwięcej tematów?\n')
    display(top_active_authors)
    
    color_discrete_sequence = ['#414fa0']*len(data)
    fig = px.bar(top_active_authors, x='author', y='count_topics', 
                labels={'author': author, 'count_topics': 'Liczba poruszonych tematów'}, color_discrete_sequence=color_discrete_sequence,
                width=500, height=350)
    
    fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)'
    })
    fig.show()

    # zapis
    pio.write_image(fig, f"{name_save}_topic_analysis.svg")
    
    return data_topic_grouped, data_topics_per_author, top_active_authors

___

## Twitter

In [4]:
data_tweets = stance_analysis('./dane/done/tweets_with_stancev2.pkl', name_save='tweets', name_author='Nazwa użytkownika')
_ = topic_analysis(data_tweets, name_save='twitter', author='Nazwa użytkownika')



W których tematach najczęściej zmieniano zdanie?





Kto najczęściej zmienia swoje poglądy?



,author,count_change_author
0,styszka,25.0
1,partiarazem,20.0
2,wojciech_krol,16.0
3,beatamk,16.0
4,pawelbejda,15.0
...,...,...
57,pikus_pol,1.0
58,l_dobrzynski,1.0
59,m_bochenek93,1.0
60,marcinwarchol,1.0




Ile czasu średnio potrzeba na zmianę poglądu?



,topic_name,mean_days_topic
18,publiczna opieka zdrowotna,185.5
17,programy mieszkaniowe,189.571429
3,bezpieczeństwo Polaków,196.638889
4,bezrobocie to wina rządu,208.8
6,działania wojskowe,220.916667
20,sądownictwo,274.271053
14,podatki,314.142857
0,NSZZ Solidarność,441.0
1,Unia Europejska,816.0
22,wsparcie osób niepełnosprawnych,951.0


Potrzeba średnio 215.85766635501923 dni na zmianę poglądu (średnia  z autorami + temat, a następnie średnia po temacie
Kto porusza najwięcej tematów?



,author,count_topics
45,m_gdula,33
64,partiarazem,33
3,agnieszkahana,30
18,donaldtusk,28
82,szymon_holownia,28
80,styszka,26
91,wojciech_krol,26
83,trzaskowski_,25
10,beatamk,25
87,waldydzikowski,25


## Sejm

In [5]:
data_sejm  = stance_analysis('./dane/done/sejm_stance_v3.pkl', name_save='sejm', name_author='Mówca', if_sejm=True)
_ = topic_analysis(data_sejm, name_save='sejm', author='Mówca')



W których tematach najczęściej zmieniano zdanie?





Kto najczęściej zmienia swoje poglądy?



,author,count_change_author
0,Krystyna Skowrońska,43.0
1,Jan Rulewski,30.0
2,Krystyna Szumilas,28.0
3,Paulina Hennig-Kloska,27.0
4,Mieczysław Augustyn,25.0
...,...,...
675,Józef Zych,1.0
676,Józef Racki,1.0
677,Julian Srebrny,1.0
678,Joanna Strzelec-Łobodzińska,1.0




Ile czasu średnio potrzeba na zmianę poglądu?



,topic_name,mean_days_topic
5,dofinansowania na mieszkania,934.041667
27,prawa kobiet,941.208333
18,komunizm,1015.0
23,ochrona prawna mniejszości narodowych,1050.0
33,rozwój rybołóstwa,1062.7
1,akcyza na alkohol,1307.625
3,bezrobocie,1327.916667
16,inwestowanie w poprawę infrastruktury drogowej,1447.0
24,paliwo,1459.1
13,hazard,1772.0


Potrzeba średnio 801.7063981772163 dni na zmianę poglądu (średnia  z autorami + temat, a następnie średnia po temacie
Kto porusza najwięcej tematów?



,author,count_topics
2067,Mirosław Suchoń,41
1967,Małgorzata Pępek,40
249,Andrzej Szlachta,39
1074,Jan Rulewski,38
496,Bogdan Borusewicz,38
2615,Tadeusz Tomaszewski,36
887,Henryk Siedlaczek,36
1847,Maria Jackowski,35
855,Grzegorz Wojciechowski,34
1313,Józef Rojek,33


## Analiza dla wybranych osób

### Porównanie jak wypowiadał się w Sejmi i na Twitterze Wojciech Król w temacie 'media publiczne'

In [6]:
def plot_stance(df, topic, name_save, auhor_label='Mówca'):
    df = df.sort_values(['date'])
    color_discrete_sequence = ['#414fa0']*len(df)
    fig = px.line(data_frame = df[df['topic_name'] == topic],
            x = 'date',
            y = 'mean_stance',
            color = 'author',
            labels = {'date': 'Data', 'stance': 'Nastawienie', 'author':auhor_label},
            markers=3, color_discrete_sequence=color_discrete_sequence,
            width=800, height=400).update_traces(line={'width':2})
    fig.update_layout(
    legend=dict(
        x=0.1, 
        y=0.9, 
        traceorder='reversed'
    ))
    
    fig.update_layout({
    'plot_bgcolor': 'rgba(240, 241, 248, 0.7)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)'
    })

    fig.update_layout(
    xaxis=dict(gridcolor='lightgray', tickfont=dict(size=13) ), 
    yaxis=dict(gridcolor='lightgray', tickfont=dict(size=13) ), 
    )

    fig.show()
    pio.write_image(fig, f"{name_save}_stance_change_in_time.svg")    

In [7]:
print("\n\nWykres zminay nastawienia w czasie dla tematu 'media publiczne' na Twitterze")
df_wk_tweets = data_tweets[data_tweets['author']=='wojciech_krol']   
plot_stance(df_wk_tweets, f'media publiczne', 'tweets', 'Nazwa użytkownika')

print("\n\nWykres zminay nastawienia w czasie dla tematu 'media publiczne' w Sejmie")
df_wk_sejm = data_sejm[data_sejm['author']=='Wojciech Król']   
plot_stance(df_wk_sejm, f'media publiczne', 'sejm', 'Mówca')



Wykres zminay nastawienia w czasie dla tematu 'media publiczne' na Twitterze




Wykres zminay nastawienia w czasie dla tematu 'media publiczne' w Sejmie


### Kto najczęściej zmienia swoje poglądy? - analiza dla 'Krystyna Skowrońska' (sejm) i 'styszka' (twitter)

In [8]:
print('styszka')
df_st_tweets = data_tweets[data_tweets['author']=='styszka'][data_tweets['stance_changed_author_topic']==True]
df_st_tweets = df_st_tweets.groupby('topic_name')['count_days'].mean().reset_index()
display(df_st_tweets)
print(f'średnia liczba dni na potrzebna na zmianę zdania {np.mean(df_st_tweets["count_days"].to_list())}')

styszka


,topic_name,count_days
0,aborcja,59.5
1,imigranci,167.666667
2,inflacja,43.0
3,konfederacja,23.181818
4,media publiczne,50.0
5,podatki,1074.0
6,programy mieszkaniowe,44.0
7,sądownictwo,196.333333


średnia liczba dni na potrzebna na zmianę zdania 207.21022727272725


In [9]:
print('Krystyna Skowrońska')
df_ks_sejm = data_sejm[data_sejm['author']=='Krystyna Skowrońska'][data_sejm['stance_changed_author_topic']==True]
df_ks_sejm = df_ks_sejm.groupby('topic_name')['count_days'].mean().reset_index()
display(df_ks_sejm)
print(f'średnia liczba dni na potrzebna na zmianę zdania {np.mean(df_ks_sejm["count_days"].to_list())}')

Krystyna Skowrońska


,topic_name,count_days
0,akcyza na wyroboy tytoniowe,126.0
1,dofinansowania na mieszkania,913.5
2,emerytura,4786.0
3,finansowanie szkolnictwa,1365.25
4,media publiczne,661.375
5,podatki,1876.5
6,publiczna opieka zdrowotna,306.111111
7,wsparcie rolników,1100.75
8,węgiel,240.666667


średnia liczba dni na potrzebna na zmianę zdania 1264.016975308642
